# Quick Example AnnoMate Reviewer

This notebook demonstrates how to run an example reviewer built on AnnoMate. 

- If you want to know more details on what AnnoMate reviewers are and extensive documentation of its features, see `Intro_to_AnnoMate_Reviewers.ipynb`
- If you want to know how to **create** your own custom reviewer, see `Developer_AnnoMate_Tutorial.ipynb`
- We recommend using pre-built reviewers if **one already exists** for a task you want to perform and/or **you consider yourself less technically experienced.** See the [AnnnoMate Reviewer Catalog](https://github.com/getzlab/AnnoMate/blob/master/catalog/README.md) for existing reviewers.

## Prerequistes:
- User is familiar with executing cells in jupyter notebooks and considers themself **beginner or intermediate level** in python
- User wants to learn some of the basic built-in functionality for customizing pre-built Annomate Reviewers

## Outcomes:
- Get a basic AnnoMate reviewer running
- Learn some of the features to update the dashboard in the notebook directly

## Installation

Option 1: Using pypi (recommended)
1. Create an environment: `conda create --name <my-env> --file requirements.txt python==3.8`
1. Activate environment: `conda activate <my-env>`
1. Install AnnoMate: `pip install AnnoMate`

Option 2: Using git
1. Download the repository: `git clone git@github.com:getzlab/AnnoMate.git` 
1. `cd AnnoMate`
1. Create an environment: `conda create --name <my-env> --file requirements.txt python==3.8`
1. Activate environment: `conda activate <my-env>`
1. Install package: `pip install -e .`

Option 3: Running from docker container. You need to open at least 2 ports if you are using Mac or Windows:
1. A port to open jupyter lab (in this case, `<jupyter_port>`)
2. A port to open the dash app (`<dash_port>`)
```
docker run -it -p <jupyter_port>:<jupyter_port> -p <dash_port>:<dash_port> ghcr.io/getzlab/annomate:latest
cd AnnoMate/tutorial_notebooks
jupyter lab --ip 0.0.0.0 --port <jupyter_port> --no-browser --allow-root
```

Finally, make sure to update your kernel to the conda environment `<my-env>` you just created. And then run the following cell (though you can skip the ***Google Colab*** section.)


In [ ]:
# set mode for dash app
mode='tab'

### If running tutorial in Google Colab
Run these cells to install AnnoMate in the Colab environment from the github source code.

*As the installed Colab packages change over time, this installation procedure is not guaranteed to work in the future.*

In [ ]:
!pip install --ignore-installed blinker==1.6.2  # gets around an installation error due to an old Colab dependency
!git clone https://github.com/getzlab/AnnoMate.git
!cd AnnoMate; pip install .

There may be some installation errors, but as long as it reports that AnnoMate was installed successfully, you should be good to go.

In [ ]:
# move into tutorial_notebooks folder to access example_data
%cd AnnoMate/tutorial_notebooks

In [ ]:
# set mode for dash app
mode='external'

## Run an Example Reviewer

Here we set up a basic reviewer that displays fake sample data. Make sure you have access to the example_data folder contained in the same directory as this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os
from AnnoMate.Reviewers.ExampleReviewer import ExampleReviewer

In [9]:
# get data
fn = 'example_data/AnnoMate_Tutorial/data_to_review_example.tsv'
df = pd.read_csv(fn, sep='\t')
df = df.set_index('sample_id')

In [27]:
# set up and run reviewer
my_reviewer = ExampleReviewer()
output_pkl_path = './example_reviewer_data'

my_reviewer.set_review_data(data_path=output_pkl_path, 
                            description='Example reviewer description',
                            sample_df=df,
                            preprocessing_str='Testing preprocessing')

my_reviewer.set_review_app(mut_file_col='mutations_file', 
                           sample_cols=['gender', 'age', 'tissue_origin'])

my_reviewer.set_default_review_data_annotations_configuration()
my_reviewer.set_default_autofill()
my_reviewer.run(collapsable=True, port=8056, mode=mode)

Setting auto_export_path to ./example_reviewer_data/data.auto_export
Using ./example_reviewer_data/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8056/


/Users/cchu/opt/anaconda3/envs/test/lib/python3.8/site-packages/AnnoMate/ReviewDataInterface.py:41: UserWarning:

Loading existing data pkl file



<IPython.core.display.Javascript object>

Accessing the Dash App:
> **If you are using a VM:** Make sure to open a ssh tunnel to the specified port number
> - `~C` to open the ssh prompt (will look like `ssh>`)
> - Type and enter `-L <port number>:localhost:<port number>`

> **If you are running from a docker container:** Make sure to open the port for accessing the dash app (`reviewer.run(port=<new_port_number>, ...)`) when you open the container
> - `docker run -it -p <jupyter_port>:<jupyter_port> -p <dash_port>:<dash_port> gcr.io/broad-getzlab-workflows/annomate:dev_branch_v385`
> - You can add more ports by adding more `-p <port_number>:<port_number>` arguments
> - If you are on Linux, you can replace `-p` arguments with `--network host`

> **If you are running in Google Colab:** All you need to do is click the link populated above (something like http://0.0.0.0:8056/), which will open another tab to display the interactive Dash App.

### Use a table to navigate between samples rather than dropdown

The table will also include the annotations in addition to the columns included in the dataframe passed to `review_data_table_df`. This table in the dashboard has native sorting and filtering, which can help the user find or prioritize samples to review.

In [30]:
my_reviewer.run(
    review_data_table_df=df[['gender', 'age', 'tissue_origin']], 
    review_data_table_page_size=7,
    port=8056,
    mode=mode
)

Setting auto_export_path to ./example_reviewer_data/data.auto_export
Using ./example_reviewer_data/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8056/


<IPython.core.display.Javascript object>

### Disable test mode

On initial load, you'll notice a `Freeze data` button. Once you are happy with your input data and are ready to start reviewing your data, press this button to disable test mode and freeze your data. Note that once this button is pressed, you will not be able to unfreeze your data / revert back to test mode.

### Retrieving annotations made in the dashbaord
After submitting annotations in the dashboard, you can view your progress by accessing the annotation table or history table:

In [35]:
my_reviewer.get_annot().head()

,Notes,Flag,Histogram color,A dropdown menu,Test removing annotation
sample_0,,Remove,,,NaN
sample_1,,Remove,,,NaN
sample_2,Nothing to say,Keep,,,NaN
sample_3,,Remove,,,NaN
sample_4,Not good,Remove,,,NaN


In [36]:
my_reviewer.get_history()

,index,timestamp,source_data_fn,Notes,Flag,Histogram color,A dropdown menu,Test removing annotation
0,sample_0,2024-05-29 11:21:53.051037,./example_reviewer_data/data.pkl,,Remove,NaN,NaN,NaN
0,sample_1,2024-05-29 11:21:56.466574,./example_reviewer_data/data.pkl,,Remove,NaN,NaN,NaN
0,sample_2,2024-05-29 11:22:02.213915,./example_reviewer_data/data.pkl,Nothing to say,Keep,NaN,NaN,NaN
0,sample_3,2024-05-29 11:22:05.466076,./example_reviewer_data/data.pkl,,Remove,NaN,NaN,NaN
0,sample_4,2024-05-29 11:22:12.998607,./example_reviewer_data/data.pkl,Not good,Remove,NaN,NaN,NaN


Now you can export this table to a file that you can then share, upload to Terra, or use for further analysis.

## Make modifications to the Dashboard in the notebook

In [37]:
from AnnoMate.Data import DataAnnotation
import AnnoMate.AnnotationDisplayComponent as adc

### Add another table component

AnnoMate includes a helper function to quickly add in a table using a tsv file provided for each sample. In this case, we display the file in column `Treatmet file` in `df` corresponding to the selected sample for review.

See `Developer_AnnoMate_Tutorial` for more details on how to make a fully custom interactive component.

In [38]:
my_reviewer.app.add_table_from_path(
    data_table_source='df', # reference which table in the Data object to use.
    table_title='Treatment file',
    component_id='treatment-component-id',
    table_fn_col='treatments_file', 
    table_cols=['treatment_name', 'response', 'tx_start']
)

### Update existing annotation options and display

By default, the `ExampleReviewer` `Flag` annotation had only two options: Keep and Remove. Here, we update the viable options for annotations, and set a default value.

In [40]:
my_reviewer.add_review_data_annotation(
    annot_name='Flag', 
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['Keep', 'Remove', 'Report'], 
    )
)
my_reviewer.add_annotation_display_component(
    annot_name='Flag', 
    annot_display_component=adc.RadioitemAnnotationDisplay(default_display_value='Keep')
)

### Add more annotations

Here we add a dropdown menu to track the annotation of the mut_vafs color, as this is not included in the pre-built ExampleReviewer class

In [42]:
# Dropdown/Select
my_reviewer.add_review_data_annotation(
    annot_name='Histogram color',
    review_data_annotation=DataAnnotation(
        annot_value_type='string', 
        options=['red', 'blue', 'green']
    )
)
my_reviewer.add_annotation_display_component(
    annot_name='Histogram color',
    annot_display_component=adc.SelectAnnotationDisplay())

### Remove annotation from display

Note this will only remove the annotation input from the dashboard. It will not delete the column in the annotation table (what is displayed when calling `reviewer.get_annot()`).

In [43]:
# adding an annotation
my_reviewer.add_review_data_annotation(
    annot_name='Test removing annotation', 
    review_data_annotation=DataAnnotation(
        annot_value_type='float', 
    )
)
my_reviewer.add_annotation_display_component(
    annot_name='Test removing annotation', 
    annot_display_component=adc.NumberAnnotationDisplay())

'Test removing annotation' in my_reviewer.annot_app_display_types_dict.keys()

True

In [44]:
# removing an annotation from display
removed_annot_display_component = my_reviewer.remove_annotation_display_component('Test removing annotation')
'Test removing annotation' in my_reviewer.annot_app_display_types_dict.keys()

False

### Add annotation autofill button

To expedite recording annotations, the user can create a button that will fill the annotation inputs. Here, when the button `Mut vafs` is clicked, the `Histogram color` annotation will be filled with whatever the current state of the radioitem `mut-figure-color-radioitem` is.

Users can also prefill with a fixed value (i.e. a string, number)

In [49]:
from dash.dependencies import State
my_reviewer.add_autofill(
    autofill_button_name='Mut vafs',
    fill_value=State('mut-figure-color-radioitem', 'value'),
    annot_name='Histogram color'
)

### Change which components and the order the components are displayed

In [50]:
# View available components
my_reviewer.app.more_components.keys()

odict_keys(['sample-info-component', 'Maf file', 'Mut vafs', 'Treatment file'])

In [51]:
my_reviewer.run(
    port=8056,
    components_name_order=['sample-info-component', 'Treatment file', 'Mut vafs'],
    mode=mode
)

Setting auto_export_path to ./example_reviewer_data/data.auto_export
Using ./example_reviewer_data/data.auto_export for auto exporting.
Dash app running on http://0.0.0.0:8056/


**NOTE**: Some reviewers may have components that listen to each other, so if you keep one but not the other, some of the predefined interactivity may lead to error messages. You may run into similar errors for autofill buttons which may be pulling data from components that are excluded. 

Also, the data needed to render excluded components will still be required when setting the review data (`reviewer.set_review_data(...)`).

# Final words

You can always go back and change the parameters of setting the app and annotation configurations, adding or editing the components of the app, etc.

The one thing that will NOT change after disabling test mode is `set_review_data()`. This review data can only be frozen once per `data_path`. You may re-set your review data as much as you'd like in test mode, but once you click the 'Freeze data' button, disabling test mode, no more changes to the input data can be made (annotaions can still be made and components can still be added). If you wish to start over, you must manually delete the data path. Else, reference a new path.

This feature allows you to make these changes to the app on the fly, and you can restart the notebook and it will not change the annotations you have already made.

